In [ ]:
%load_ext autoreload
%autoreload 2

from imports import *
from models import *
from utils import *
from data import *
from configs import CONFIGS, EXP_CODES

plt.rcParams.update({'font.size': 22})

In [ ]:
'''
load dataset
'''


config_name = 'bird2d'
config = deepcopy(CONFIGS[config_name])

train_dataset, test_dataset, train_loader, test_loader = get_dataset_dataloader(config)

### True space

In [ ]:
config_name = 'bird2d'
config = deepcopy(CONFIGS[config_name])


'''
generate trajectory
'''
trajectory, gt_vs = test_dataset.generate_sample_trajectory(length=10000)

trajectory = trajectory.squeeze(dim=1)
gt_velocities = np.array(gt_vs)
colors = np.array(test_dataset.cmap(gt_vs))

assert gt_velocities.ndim == 2

# pad gt_velocities with necessary zeros
pad = np.zeros((gt_velocities.shape[0], config['model_args']['v_dim'] - gt_velocities.shape[1]))
gt_velocities = np.hstack((gt_velocities, pad))

if np.prod(pad.shape) == 0 and config['task_dim'] == 1:
    gt_velocities = np.hstack((gt_velocities, np.zeros((gt_velocities.shape[0], 1))))



plt.rcParams.update({'font.size': 22})

plt.figure()

plt.scatter(*gt_velocities.T, c=colors, s=300)

ax = plt.gca()
# ax.set_xlim([-1.75, 1.75])
# ax.set_ylim([-1.75, 1.75])
ax.set_xlabel('dim 1')
ax.set_ylabel('dim 2')
# ax.set_title('True space\n')

ax.xaxis.set_tick_params(labelbottom=False)
ax.yaxis.set_tick_params(labelleft=False)
ax.set_xticks([])
ax.set_yticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('./figures/bird2d_true_space.png')

### Our results

In [ ]:
config_name = 'bird2d'
config = deepcopy(CONFIGS[config_name])


'''
generate trajectory
'''
trajectory, gt_vs = test_dataset.generate_sample_trajectory(length=5000)

trajectory = trajectory.squeeze(dim=1)
gt_velocities = np.array(gt_vs)
colors = np.array(test_dataset.cmap(gt_vs))

assert gt_velocities.ndim == 2

# pad gt_velocities with necessary zeros
pad = np.zeros((gt_velocities.shape[0], config['model_args']['v_dim'] - gt_velocities.shape[1]))
gt_velocities = np.hstack((gt_velocities, pad))

if np.prod(pad.shape) == 0 and config['task_dim'] == 1:
    gt_velocities = np.hstack((gt_velocities, np.zeros((gt_velocities.shape[0], 1))))


'''
evaluate method
'''
pred_velocities_ours, transformed_pred_velocities_ours, fit_errors_ours, firing_rates_ours = [], [], [], []

for model_code in EXP_CODES[config_name]:
    pred_velocities = our_reduction(
        trajectory,
        config,
        model_code,
    )

    error, transformed_pred_velocities = compute_error_metric(
        gt_velocities,
        pred_velocities,
    )

    plt.ioff()
    fig = firing_rates(
        config,
        gt_velocities,
        transformed_pred_velocities,
        base_grid_period=0.2,
        resample_every=10,
    )
    plt.close(fig)
    plt.ion()

    pred_velocities_ours.append(pred_velocities)
    transformed_pred_velocities_ours.append(transformed_pred_velocities)
    fit_errors_ours.append(error)
    firing_rates_ours.append(fig)

In [ ]:
firing_rates_ours[1].savefig('./figures/bird2d_ours_firing_rates.png')

In [ ]:
fig = plt.figure(figsize=(6, 6))

ax = plt.gca()


ax.scatter(*pred_velocities_ours[3].T, c=colors, s=100)

ax.set_xlabel('dim 1')
ax.set_ylabel('dim 2')
# ax.set_title('Our model')

ax.xaxis.set_tick_params(labelbottom=False)
ax.yaxis.set_tick_params(labelleft=False)
ax.set_xticks([])
ax.set_yticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('./figures/bird2d_ours.png')

In [ ]:
config_name = 'bird2d_3dim'
config = deepcopy(CONFIGS[config_name])


'''
generate trajectory
'''
trajectory, gt_vs = test_dataset.generate_sample_trajectory(length=5000)

trajectory = trajectory.squeeze(dim=1)
gt_velocities = np.array(gt_vs)
colors = np.array(test_dataset.cmap(gt_vs))

assert gt_velocities.ndim == 2

# pad gt_velocities with necessary zeros
pad = np.zeros((gt_velocities.shape[0], config['model_args']['v_dim'] - gt_velocities.shape[1]))
gt_velocities = np.hstack((gt_velocities, pad))

if np.prod(pad.shape) == 0 and config['task_dim'] == 1:
    gt_velocities = np.hstack((gt_velocities, np.zeros((gt_velocities.shape[0], 1))))


'''
evaluate method
'''
pred_velocities_3dim_ours, transformed_pred_velocities_3dim_ours, fit_errors_3dim_ours = [], [], []

for model_code in EXP_CODES[config_name]:
    pred_velocities = our_reduction(
        trajectory,
        config,
        model_code,
    )

    error, transformed_pred_velocities = compute_error_metric(
        gt_velocities,
        pred_velocities,
    )

    pred_velocities_3dim_ours.append(pred_velocities)
    transformed_pred_velocities_3dim_ours.append(transformed_pred_velocities)
    fit_errors_3dim_ours.append(error)

In [ ]:
fig = plt.figure(figsize=(6, 6))

ax = fig.add_subplot(projection='3d')

ax.scatter3D(*pred_velocities_3dim_ours[2].T, c=colors, s=100)

# ax.set_xlim([-0.07, 0.07])
# ax.set_ylim([-0.07, 0.07])
# ax.set_zlim([-0.07, 0.07])
ax.set_xlabel('dim 1')
ax.set_ylabel('dim 2')
ax.set_zlabel('dim 3')
# ax.set_title('Our model')

ax.xaxis.set_tick_params(labelbottom=False)
ax.yaxis.set_tick_params(labelleft=False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_zticks([])
ax.xaxis.labelpad=-11.5
ax.yaxis.labelpad=-11.5
ax.zaxis.labelpad=-11.5

ax.view_init(elev=40, azim=160.0)

plt.tight_layout()
plt.savefig('./figures/bird2d_3dim_ours.png')

In [ ]:
explained_variance(pred_velocities_3dim_ours[2])

### PCA results

In [ ]:
config_name = 'bird2d'
config = deepcopy(CONFIGS[config_name])


'''
generate trajectory
'''
trajectory, gt_vs = test_dataset.generate_sample_trajectory(length=5000)

trajectory = trajectory.squeeze(dim=1)
gt_velocities = np.array(gt_vs)
colors = np.array(test_dataset.cmap(gt_vs))

assert gt_velocities.ndim == 2

# pad gt_velocities with necessary zeros
pad = np.zeros((gt_velocities.shape[0], config['model_args']['v_dim'] - gt_velocities.shape[1]))
gt_velocities = np.hstack((gt_velocities, pad))

if np.prod(pad.shape) == 0 and config['task_dim'] == 1:
    gt_velocities = np.hstack((gt_velocities, np.zeros((gt_velocities.shape[0], 1))))



pca_pred_velocities = pca_reduction(
    trajectory,
    config,
    2,
)

fit_error_pca, pca_transformed_pred_velocities = compute_error_metric(
    gt_velocities,
    pca_pred_velocities,
)

plt.ioff()
pca_firing_rates = firing_rates(
    config,
    gt_velocities,
    pca_transformed_pred_velocities,
    base_grid_period=0.2,
    resample_every=10,
)
plt.close(pca_firing_rates)
plt.ion()

In [ ]:
pca_firing_rates.savefig('./figures/bird2d_pca_firing_rates.png')

In [ ]:
fig = plt.figure(figsize=(6, 6))

ax = plt.gca()


ax.scatter(*pca_pred_velocities.T, c=colors, s=100)

ax.set_xlabel('dim 1')
ax.set_ylabel('dim 2')
# ax.set_title('PCA')

ax.xaxis.set_tick_params(labelbottom=False)
ax.yaxis.set_tick_params(labelleft=False)
ax.set_xticks([])
ax.set_yticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('./figures/bird2d_pca.png')

### Isomap results

In [ ]:
config_name = 'bird2d'
config = deepcopy(CONFIGS[config_name])


'''
generate trajectory
'''
trajectory, gt_vs = test_dataset.generate_sample_trajectory(length=5000)

trajectory = trajectory.squeeze(dim=1)
gt_velocities = np.array(gt_vs)
colors = np.array(test_dataset.cmap(gt_vs))

assert gt_velocities.ndim == 2

# pad gt_velocities with necessary zeros
pad = np.zeros((gt_velocities.shape[0], config['model_args']['v_dim'] - gt_velocities.shape[1]))
gt_velocities = np.hstack((gt_velocities, pad))

if np.prod(pad.shape) == 0 and config['task_dim'] == 1:
    gt_velocities = np.hstack((gt_velocities, np.zeros((gt_velocities.shape[0], 1))))



isomap_pred_velocities = isomap_reduction(
    trajectory,
    config,
    2,
)

fit_error_isomap, isomap_transformed_pred_velocities = compute_error_metric(
    gt_velocities,
    isomap_pred_velocities,
)

plt.ioff()
isomap_firing_rates = firing_rates(
    config,
    gt_velocities,
    isomap_transformed_pred_velocities,
    base_grid_period=0.2,
    resample_every=10,
)
plt.close(isomap_firing_rates)
plt.ion()

In [ ]:
isomap_firing_rates.savefig('./figures/bird2d_isomap_firing_rates.png')

In [ ]:
fig = plt.figure(figsize=(6, 6))

ax = plt.gca()


ax.scatter(*isomap_pred_velocities.T, c=colors, s=100)

ax.set_xlabel('dim 1')
ax.set_ylabel('dim 2')
# ax.set_title('Isomap')

ax.xaxis.set_tick_params(labelbottom=False)
ax.yaxis.set_tick_params(labelleft=False)
ax.set_xticks([])
ax.set_yticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('./figures/bird2d_isomap.png')

### UMAP results

In [ ]:
config_name = 'bird2d'
config = deepcopy(CONFIGS[config_name])


'''
generate trajectory
'''
trajectory, gt_vs = test_dataset.generate_sample_trajectory(length=5000)

trajectory = trajectory.squeeze(dim=1)
gt_velocities = np.array(gt_vs)
colors = np.array(test_dataset.cmap(gt_vs))

assert gt_velocities.ndim == 2

# pad gt_velocities with necessary zeros
pad = np.zeros((gt_velocities.shape[0], config['model_args']['v_dim'] - gt_velocities.shape[1]))
gt_velocities = np.hstack((gt_velocities, pad))

if np.prod(pad.shape) == 0 and config['task_dim'] == 1:
    gt_velocities = np.hstack((gt_velocities, np.zeros((gt_velocities.shape[0], 1))))



umap_pred_velocities = umap_reduction(
    trajectory,
    config,
    2,
)

fit_error_umap, umap_transformed_pred_velocities = compute_error_metric(
    gt_velocities,
    umap_pred_velocities,
)

plt.ioff()
umap_firing_rates = firing_rates(
    config,
    gt_velocities,
    umap_transformed_pred_velocities,
    base_grid_period=0.2,
    resample_every=10,
)
plt.close(umap_firing_rates)
plt.ion()

In [ ]:
umap_firing_rates.savefig('./figures/bird2d_umap_firing_rates.png')

In [ ]:
fig = plt.figure(figsize=(6, 6))

ax = plt.gca()


ax.scatter(*umap_pred_velocities.T, c=colors, s=100)

ax.set_xlabel('dim 1')
ax.set_ylabel('dim 2')
# ax.set_title('UMAP')

ax.xaxis.set_tick_params(labelbottom=False)
ax.yaxis.set_tick_params(labelleft=False)
ax.set_xticks([])
ax.set_yticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('./figures/bird2d_umap.png')

### Autoencoder results

In [ ]:
config_name = 'bird2d_ae'
config = deepcopy(CONFIGS[config_name])


'''
generate trajectory
'''
trajectory, gt_vs = test_dataset.generate_sample_trajectory(length=5000)

trajectory = trajectory.squeeze(dim=1)
gt_velocities = np.array(gt_vs)
colors = np.array(test_dataset.cmap(gt_vs))

assert gt_velocities.ndim == 2

# pad gt_velocities with necessary zeros
pad = np.zeros((gt_velocities.shape[0], config['model_args']['v_dim'] - gt_velocities.shape[1]))
gt_velocities = np.hstack((gt_velocities, pad))

if np.prod(pad.shape) == 0 and config['task_dim'] == 1:
    gt_velocities = np.hstack((gt_velocities, np.zeros((gt_velocities.shape[0], 1))))


'''
evaluate method
'''
pred_velocities_ae, transformed_pred_velocities_ae, fit_errors_ae, firing_rates_ae = [], [], [], []

for model_code in EXP_CODES[config_name]:
    pred_velocities = ae_reduction(
        trajectory,
        config,
        model_code,
    )

    error, transformed_pred_velocities = compute_error_metric(
        gt_velocities,
        pred_velocities,
    )

    plt.ioff()
    fig = firing_rates(
        config,
        gt_velocities,
        transformed_pred_velocities,
        base_grid_period=0.2,
        resample_every=10,
    )
    plt.close(fig)
    plt.ion()

    pred_velocities_ae.append(pred_velocities)
    transformed_pred_velocities_ae.append(transformed_pred_velocities)
    fit_errors_ae.append(error)
    firing_rates_ae.append(fig)

In [ ]:
firing_rates_ae[0].savefig('./figures/bird2d_ae_firing_rates.png')

In [ ]:
fig = plt.figure(figsize=(6, 6))

ax = plt.gca()


ax.scatter(*pred_velocities_ae[0].T, c=colors, s=100)

ax.set_xlabel('dim 1')
ax.set_ylabel('dim 2')
# ax.set_title('Autoencoder')

ax.xaxis.set_tick_params(labelbottom=False)
ax.yaxis.set_tick_params(labelleft=False)
ax.set_xticks([])
ax.set_yticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('./figures/bird2d_ae.png')

### MCNet results

In [ ]:
config_name = 'bird2d_mcnet'
config = deepcopy(CONFIGS[config_name])


'''
generate trajectory
'''
trajectory, gt_vs = test_dataset.generate_sample_trajectory(length=5000)

trajectory = trajectory.squeeze(dim=1)
gt_velocities = np.array(gt_vs)
colors = np.array(test_dataset.cmap(gt_vs))

assert gt_velocities.ndim == 2

# pad gt_velocities with necessary zeros
pad = np.zeros((gt_velocities.shape[0], config['model_args']['v_dim'] - gt_velocities.shape[1]))
gt_velocities = np.hstack((gt_velocities, pad))

if np.prod(pad.shape) == 0 and config['task_dim'] == 1:
    gt_velocities = np.hstack((gt_velocities, np.zeros((gt_velocities.shape[0], 1))))


'''
evaluate method
'''
pred_velocities_mcnet, transformed_pred_velocities_mcnet, fit_errors_mcnet, firing_rates_mcnet = [], [], [], []

for model_code in EXP_CODES[config_name]:
    pred_velocities = mcnet_reduction(
        trajectory,
        config,
        model_code,
    )

    error, transformed_pred_velocities = compute_error_metric(
        gt_velocities,
        pred_velocities,
    )

    plt.ioff()
    fig = firing_rates(
        config,
        gt_velocities,
        transformed_pred_velocities,
        base_grid_period=0.2,
        resample_every=10,
    )
    plt.close(fig)
    plt.ion()

    pred_velocities_mcnet.append(pred_velocities)
    transformed_pred_velocities_mcnet.append(transformed_pred_velocities)
    fit_errors_mcnet.append(error)
    firing_rates_mcnet.append(fig)

In [ ]:
firing_rates_mcnet[0].savefig('./figures/bird2d_mcnet_firing_rates.png')

In [ ]:
fig = plt.figure(figsize=(6, 6))

ax = plt.gca()


ax.scatter(*pred_velocities_mcnet[0].T, c=colors, s=100)

ax.set_xlabel('dim 1')
ax.set_ylabel('dim 2')
# ax.set_title('MCNet')

ax.xaxis.set_tick_params(labelbottom=False)
ax.yaxis.set_tick_params(labelleft=False)
ax.set_xticks([])
ax.set_yticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('./figures/bird2d_mcnet.png')

In [ ]:
errors = [fit_errors_ours, fit_errors_mcnet, fit_errors_ae, [fit_error_pca], [fit_error_isomap], [fit_error_umap]]

# Create a box plot
fig, ax = plt.subplots(figsize=(9, 5))

# Plot the box plot for experiments with multiple trials
boxplot_data = [e for e in errors if len(e) > 1]
ax.boxplot(boxplot_data, positions=[0, 1, 2])

# Plot the single trial errors as points
single_trial_positions = [3, 4, 5]
single_trial_errors = [e[0] for e in errors if len(e) == 1]
ax.plot(single_trial_positions, single_trial_errors, 'ro')  # 'ro' is for red dots

# Adjust x-axis labels to match the experiments
ax.set_xticks(range(len(errors)))
ax.set_xticklabels(['Ours', 'MCNet', 'AE', 'PCA', 'Isomap', 'UMAP'])

# Set labels and title
ax.set_ylabel('Error')
ax.set_yscale('log')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('./figures/bird2d_results.pdf')

In [ ]:
def round_up(value):
    return math.ceil(value * 100) / 100

labels = ['ours', 'mcnet', 'ae', 'pca', 'isomap', 'umap']

for label, error in zip(labels, errors):
    print("{}: {:.2f} $\pm$ {:.2f}".format(label, round_up(np.mean(error)), round_up(np.std(error))))

### Ablations

In [ ]:
# config_name = 'bird2d_ablation_image_loss'
# config = deepcopy(CONFIGS[config_name])
# model_code = EXP_CODES[config_name][0]

# '''
# generate trajectory
# '''
# trajectory, gt_vs = test_dataset.generate_sample_trajectory(length=5000)

# trajectory = trajectory.squeeze(dim=1)
# gt_velocities = np.array(gt_vs)
# colors = np.array(test_dataset.cmap(gt_vs))

# assert gt_velocities.ndim == 2

# # pad gt_velocities with necessary zeros
# pad = np.zeros((gt_velocities.shape[0], config['model_args']['v_dim'] - gt_velocities.shape[1]))
# gt_velocities = np.hstack((gt_velocities, pad))

# if np.prod(pad.shape) == 0 and config['task_dim'] == 1:
#     gt_velocities = np.hstack((gt_velocities, np.zeros((gt_velocities.shape[0], 1))))


# '''
# evaluate method
# '''

# pred_velocities_image_loss_ablation = our_reduction(
#     trajectory,
#     config,
#     model_code,
# )

# error, transformed_pred_velocities_image_loss_ablation = compute_error_metric(
#     gt_velocities,
#     pred_velocities_image_loss_ablation,
# )

# plt.ioff()
# firing_rates_image_loss_ablation = firing_rates(
#     config,
#     gt_velocities,
#     transformed_pred_velocities_image_loss_ablation,
#     base_grid_period=0.2,
#     resample_every=10,
# )
# plt.close(firing_rates_image_loss_ablation)
# plt.ion()

In [ ]:
# firing_rates_image_loss_ablation.savefig('./figures/bird2d_firing_rates_image_loss_ablation.png')

In [ ]:
# fig = plt.figure(figsize=(6, 6))

# ax = plt.gca()


# ax.scatter(*pred_velocities_image_loss_ablation.T, c=colors, s=100)

# ax.set_xlabel('dim 1')
# ax.set_ylabel('dim 2')
# ax.set_title('State Prediction Loss')

# ax.xaxis.set_tick_params(labelbottom=False)
# ax.yaxis.set_tick_params(labelleft=False)
# ax.set_xticks([])
# ax.set_yticks([])
# ax.spines['top'].set_visible(False)
# ax.spines['right'].set_visible(False)

# plt.tight_layout()
# plt.savefig('./figures/bird2d_image_loss_ablation.png')

In [ ]:
# config_name = 'bird2d_ablation_image_and_loop_loss'
# config = deepcopy(CONFIGS[config_name])
# model_code = EXP_CODES[config_name][0]

# '''
# generate trajectory
# '''
# trajectory, gt_vs = test_dataset.generate_sample_trajectory(length=5000)

# trajectory = trajectory.squeeze(dim=1)
# gt_velocities = np.array(gt_vs)
# colors = np.array(test_dataset.cmap(gt_vs))

# assert gt_velocities.ndim == 2

# # pad gt_velocities with necessary zeros
# pad = np.zeros((gt_velocities.shape[0], config['model_args']['v_dim'] - gt_velocities.shape[1]))
# gt_velocities = np.hstack((gt_velocities, pad))

# if np.prod(pad.shape) == 0 and config['task_dim'] == 1:
#     gt_velocities = np.hstack((gt_velocities, np.zeros((gt_velocities.shape[0], 1))))


# '''
# evaluate method
# '''

# pred_velocities_image_and_loop_loss_ablation = our_reduction(
#     trajectory,
#     config,
#     model_code,
# )

# error, transformed_pred_velocities_image_and_loop_loss_ablation = compute_error_metric(
#     gt_velocities,
#     pred_velocities_image_and_loop_loss_ablation,
# )

# plt.ioff()
# firing_rates_image_and_loop_loss_ablation = firing_rates(
#     config,
#     gt_velocities,
#     transformed_pred_velocities_image_and_loop_loss_ablation,
#     base_grid_period=0.2,
#     resample_every=10,
# )
# plt.close(firing_rates_image_and_loop_loss_ablation)
# plt.ion()

In [ ]:
# firing_rates_image_and_loop_loss_ablation.savefig('./figures/bird2d_firing_rates_image_and_loop_loss_ablation.png')

In [ ]:
# fig = plt.figure(figsize=(6, 6))

# ax = plt.gca()


# ax.scatter(*pred_velocities_image_and_loop_loss_ablation.T, c=colors, s=100)

# ax.set_xlabel('dim 1')
# ax.set_ylabel('dim 2')
# ax.set_title('State Prediction and Loop Loss')

# ax.xaxis.set_tick_params(labelbottom=False)
# ax.yaxis.set_tick_params(labelleft=False)
# ax.set_xticks([])
# ax.set_yticks([])
# ax.spines['top'].set_visible(False)
# ax.spines['right'].set_visible(False)

# plt.tight_layout()
# plt.savefig('./figures/bird2d_image_and_loop_loss_ablation.png')